<a href="https://colab.research.google.com/github/NayraSousa/mrfi-teste/blob/dev/lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mrfi
!pip install torch
!pip install torchvision

In [3]:
from mrfi import MRFI, EasyConfig
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from mrfi.experiment import Acc_experiment, Acc_golden

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class LeNet(nn.Module):
  def __init__(self, trained=False):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*4*4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

    if trained:
      self.load_state_dict(torch.load('/content/drive/MyDrive/LeNet/train/lenet_mnist.pth'))

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(x.size()[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

lenet = LeNet()

In [5]:
def get_mnist(batch_size=64):
  transform = transforms.Compose(
      [transforms.ToTensor(),
       transforms.Normalize((0.1307,), (0.3081,))]
  )

  trainset = torchvision.datasets.MNIST(
      root='/content/drive/MyDrive/LeNet/data', train=True, download=True, transform=transform
  )
  testset = torchvision.datasets.MNIST(
      root='/content/drive/MyDrive/LeNet/data', train=False, download=True, transform=transform
  )

  trainloader = torch.utils.data.DataLoader(
      trainset, batch_size=batch_size, shuffle=True
  )
  testloader = torch.utils.data.DataLoader(
      testset, batch_size=batch_size, shuffle=False
  )

  return trainloader, testloader, testset

In [6]:
def load_dataset(dataset='mnist', batch_size=64):

  if dataset.lower()=='mnist':
    train, test, testset = get_mnist()
    return train, test, testset

trainloader, testloader,testset = load_dataset()

In [7]:
def train(epochs=3):
  lenet.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(lenet.parameters(), lr=0.001)

  lenet.train()

  for epoch in range(epochs):
    running_loss = 0
    batch_size = 100
    for i, data in trainloader:
      inputs, labels = i.to(device), data.to(device)

      optimizer.zero_grad()

      outputs = lenet(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}")
  torch.save(lenet.state_dict(), '/content/drive/MyDrive/LeNet/train/lenet_mnist.pth')
train()

Epoch 1/3, Loss: 0.2322
Epoch 2/3, Loss: 0.0720
Epoch 3/3, Loss: 0.0536


In [8]:
def test():
  lenet = LeNet().to(device)
  lenet.load_state_dict(torch.load('/content/drive/MyDrive/LeNet/train/lenet_mnist.pth'))
  lenet.eval()

  correct = 0
  total = 0

  with torch.no_grad():
      for images, labels in testloader:
          images, labels = images.to(device), labels.to(device)
          outputs = lenet(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          break

      print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

test()

Accuracy of the network on the 10000 test images: 98 %


In [9]:
for name, param in lenet.named_parameters():
    print(f"Camada: {name}")
    print(f"Parâmetros: {param.count_nonzero()}")
    print("-" * 50)

Camada: conv1.weight
Parâmetros: 150
--------------------------------------------------
Camada: conv1.bias
Parâmetros: 6
--------------------------------------------------
Camada: conv2.weight
Parâmetros: 2400
--------------------------------------------------
Camada: conv2.bias
Parâmetros: 16
--------------------------------------------------
Camada: fc1.weight
Parâmetros: 30720
--------------------------------------------------
Camada: fc1.bias
Parâmetros: 120
--------------------------------------------------
Camada: fc2.weight
Parâmetros: 10080
--------------------------------------------------
Camada: fc2.bias
Parâmetros: 84
--------------------------------------------------
Camada: fc3.weight
Parâmetros: 840
--------------------------------------------------
Camada: fc3.bias
Parâmetros: 10
--------------------------------------------------


In [10]:
import math

def calculo(e, N, t):

  denominador = 1 + (e ** 2) * ((N-1) / ((t**2) * 0.25))
  N_inj = N / denominador
  return N_inj

n_conv1 = calculo(0.01, 150, 2.60)
n_fc = calculo(0.01, 840, 2.60)

In [11]:
n_conv1

148.68907267288404

In [12]:
n_fc

800.2705902249281

In [13]:
config_str = """
faultinject:
  - type: weights
    name: [weight]
    quantization:
      method: SymmericQuantization
      bit_width: 8
      dynamic_range: auto
    selector:
      method: RandomPositionByNumber
      n: 800
      per_instance: True
    error_mode:
      method: IntRandomBitFlip
      bit_width: 8
    module_name: fc3
  - type: weights
    name: [weight]
    quantization:
      method: SymmericQuantization
      bit_width: 8
      dynamic_range: auto
    selector:
      method: RandomPositionByNumber
      n: 150
      per_instance: True
    error_mode:
      method: IntRandomBitFlip
      bit_width: 8
    module_name: conv1

"""

econfig = EasyConfig.load_string(config_str)

In [14]:
fi_model = MRFI(lenet.eval(), econfig)
fi_model.to(device)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [15]:
cfg = fi_model.get_weights_configs("selector")
print(cfg)

[<'FI_STAGE' node 'model.sub_modules.conv1.weights.0.selector'>, <'FI_STAGE' node 'model.sub_modules.fc3.weights.0.selector'>]


In [16]:
for i in range(len(cfg)):
    acc_golden, acc_fi = 0, 0
    cfg.enabled = False
    cfg[i].enabled = True
    fi_model.observers_reset()
    for images, labels in testloader:
        images=images.to(device)
        labels = labels.to(device)


        with fi_model.golden_run():
            outs_golden=fi_model(images).to(device)

        outs_fi=fi_model(images).to(device)

        acc_golden+=(outs_golden.argmax(1)==labels).sum().item()
        acc_fi+=(outs_fi.argmax(1)==labels).sum().item()

    print(f'{len(testloader.dataset)} images, acc_golden {acc_golden}, acc_fi {acc_fi}')
    print(fi_model.observers_result())

10000 images, acc_golden 9863, acc_fi 6243
{}
10000 images, acc_golden 9863, acc_fi 6257
{}
